# Starting off:
## Transform your parquet file to csv: (Change CSV name accordingly.)
# Use: prqt_to_csv.py

In [3]:
import pandas as pd

df = pd.read_parquet('yellow_tripdata_2020-01.parquet')
df.to_csv('tripdata.csv')

# Separating the files.
## Stores filtered rows by dates: (have a main_file *transformed CSV)

In [ ]:
import csv
import collections

src_path = ""
main_file = "./tripdata.csv."  # Your csv file here
choose_filter_date = "2020-01"

# Opens the file and sorts the relevant/invalid dates.
with open(main_file, "rt") as fp:
    root = csv.reader(fp, delimiter=",")
    result = collections.defaultdict(list)
    invalid = collections.defaultdict(list)
    next(root)
    for row in root:
        date = row[3].split(" ")
        if date[0].startswith(choose_filter_date + "-"):  # Choose_filter_date f.e. =2020-02
            result[date[0]].append(row)
        else:
            invalid[date[0]].append(row)

## Stores column values to be used for each new file:

In [8]:
import csv

# Grabs the column names, to later inject into new files
with open('./test.csv') as file:
    csv_reader = csv.reader(file, delimiter=",")
    col_names = []
    for row in csv_reader:
        col_names.append(row)
        break

## Creates the new files: (res= choose what you want written *result or invalid) (pth=your pathname)

In [1]:
# Creates files using the results and your chosen path
def write_files(res, pth):
    for i, j in res.items():  # res= result or invalid
        file_path = "./" + pth + "/%s%s.csv" % (src_path, i)  # Changes pth to where you will store CSV files
        with open(file_path, 'wt') as new_file:
            writer = csv.writer(new_file, delimiter=",")
            writer.writerows(col_names)
            writer.writerows(j)


## Run the functions with appropraite params:

In [ ]:
write_files(invalid, "invalid_dates")
write_files(result, "january")

# Loader.py:
## Loads your chosen files into SQLite (choose path and file at bottom)

In [ ]:
import csv
import sqlite3
import glob
import os


def load_directory(dirname, db):
    for filename in glob.glob(os.path.join(dirname, '*.csv')):
        load_file(filename, db)

def load_file(filename, db):
        with open(filename) as file:
            with db:
                data = csv.DictReader(file)
                cols = data.fieldnames
                table = os.path.splitext(os.path.basename(filename))[0]

                sql = 'drop table if exists "{}"'.format(table)
                db.execute(sql)

                sql = 'create table "{table}" ( {cols} )'.format(
                    table=table,
                    cols=','.join('"{}"'.format(col) for col in cols))
                db.execute(sql)

                sql = 'insert into "{table}" values ( {vals} )'.format(
                    table=table,
                    vals=','.join('?' for col in cols))
                db.executemany(sql, (list(map(row.get, cols)) for row in data))

if __name__ == '__main__':
    conn = sqlite3.connect('january_trips.db')
    load_directory('./january', conn)
    
    
# Optional for injection of invalid_dates

# if __name__ == '__main__':
#     conn = sqlite3.connect('invalid.db')
#     load_directory('./invalid_dates', conn)